In [3]:
import casadi
import matplotlib.pyplot as plt
import numpy as np
import sympy as sp
from IPython.display import HTML, display
from matplotlib.animation import FuncAnimation

##### Derive dynamics

In [57]:
L_body = 1
L_leg_upper = 0.4
L_leg_lower = 0.4

mb1 = 1
mb2 = 1
mb3 = 1
mb4 = 1
mb5 = 1
g = 9.81

t = sp.symbols("t")
x, y, phi, theta1, theta2, theta3, theta4 = [sp.Function(name)(t) for name in "x, y, phi, theta1, theta2, theta3, theta4".split(", ")]

x_pos_b1 = x
y_pos_b1 = y

x_front_shoulder = x_pos_b1 + (L_body / 2 * sp.cos(phi))    
y_front_shoulder = y_pos_b1 + (L_body / 2 * sp.sin(phi))

x_pos_b2 = x_front_shoulder + (L_leg_upper * sp.cos(phi - np.pi / 2 + theta1))
y_pos_b2 = y_front_shoulder + (L_leg_upper * sp.sin(phi - np.pi / 2 + theta1))

x_pos_b3 = x_pos_b2 + (L_leg_lower * sp.cos(phi - np.pi / 2 + theta1 + theta2))
y_pos_b3 = y_pos_b2 + (L_leg_lower * sp.sin(phi - np.pi / 2 + theta1 + theta2))

x_rear_shoulder = x_pos_b1 - (L_body / 2 * sp.cos(phi))
y_rear_shoulder = y_pos_b1 - (L_body / 2 * sp.sin(phi))

x_pos_b4 = x_rear_shoulder + (L_leg_upper * sp.cos(phi - np.pi / 2 + theta3))
y_pos_b4 = y_rear_shoulder + (L_leg_upper * sp.sin(phi - np.pi / 2 + theta3))

x_pos_b5 = x_pos_b4 + (L_leg_lower * sp.cos(phi - np.pi / 2 + theta3 + theta4))
y_pos_b5 = y_pos_b4 + (L_leg_lower * sp.sin(phi - np.pi / 2 + theta3 + theta4))

x_dot, y_dot, phi_dot, theta1_dot, theta2_dot, theta3_dot, theta4_dot = [sp.diff(var, t) for var in [x, y, phi, theta1, theta2, theta3, theta4]]

x_pos_b1_dot = sp.diff(x_pos_b1, t)
y_pos_b1_dot = sp.diff(y_pos_b1, t)

x_pos_b2_dot = sp.diff(x_pos_b2, t)
y_pos_b2_dot = sp.diff(y_pos_b2, t)

x_pos_b3_dot = sp.diff(x_pos_b3, t)
y_pos_b3_dot = sp.diff(y_pos_b3, t)

x_pos_b4_dot = sp.diff(x_pos_b4, t)
y_pos_b4_dot = sp.diff(y_pos_b4, t)

x_pos_b5_dot = sp.diff(x_pos_b5, t)
y_pos_b5_dot = sp.diff(y_pos_b5, t)

xs, ys, phis, theta1s, theta2s, theta3s, theta4s = [sp.Symbol(name) for name in "x, y, phi, theta1, theta2, theta3, theta4".split(", ")]
xs_dot, ys_dot, phis_dot, theta1s_dot, theta2s_dot, theta3s_dot, theta4s_dot = [sp.Symbol(name + "_dot") for name in "x, y, phi, theta1, theta2, theta3, theta4".split(", ")]

In [73]:
T = sum(1/2 * m * (xvel ** 2 + yvel ** 2) for m, xvel, yvel in [(mb1, x_pos_b1_dot, y_pos_b1_dot), (mb2, x_pos_b2_dot, y_pos_b2_dot), (mb3, x_pos_b3_dot, y_pos_b3_dot), (mb4, x_pos_b4_dot, y_pos_b4_dot), (mb5, x_pos_b5_dot, y_pos_b5_dot)])
U = sum(m * g * y for m, y in [(mb1, y_pos_b1), (mb2, y_pos_b2), (mb3, y_pos_b3), (mb4, y_pos_b4), (mb5, y_pos_b5)])
L = T - U

# Derive the equations of motion
# Create a list of generalized coordinates and their derivatives
q = [x, y, phi, theta1, theta2, theta3, theta4] 
q_dot = [x_dot, y_dot, phi_dot, theta1_dot, theta2_dot, theta3_dot, theta4_dot]
qs = [xs, ys, phis, theta1s, theta2s, theta3s, theta4s]
qs_dot = [xs_dot, ys_dot, phis_dot, theta1s_dot, theta2s_dot, theta3s_dot, theta4s_dot]

# Calculate equations of motion using Lagrange's equations
eqns = []
for i in range(len(q)):
    # Calculate d/dt(∂L/∂q̇) - ∂L/∂q = 0
    dL_dq_dot = sp.diff(L, q_dot[i])
    ddt_dL_dq_dot = sp.diff(dL_dq_dot, t)
    dL_dq = sp.diff(L, q[i])
    eqn = ddt_dL_dq_dot - dL_dq
    for i in range(len(q_dot)):
        eqn = eqn.subs(q_dot[i], qs_dot[i])

    for i in range(len(q)):
        eqn = eqn.subs(q[i], qs[i])

    eqns.append(eqn)


